# Logistic Regression

#### 모듈 불러오기

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

#### 파라메터 설정

In [ ]:
# Hyper-parameters
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

#### MNIST 데이터셋으로 데이터로더 구성

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

#### 데이터 확인

In [ ]:
train_dataset

In [ ]:
# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(images.shape)

In [ ]:
print(images[0])

In [ ]:
# functions to show an image
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(labels)

#### Logistic regression model

In [ ]:
model = nn.Linear(input_size, num_classes)

#### Loss and optimizer


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#### Train the model

In [ ]:
total_step = len(train_loader)
loss_values = []

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_values.append(loss.item())

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


In [ ]:
#학습 그래프 출력

fig, loss_ax = plt.subplots()

loss_ax.plot(loss_values, 'b', label='loss',  fillstyle='none')

loss_ax.set_xlabel('step')
loss_ax.set_ylabel('Loss')
fig.suptitle('Loss of Logistic Regression')

loss_ax.legend(loc='lower right')


plt.show()

#### Test the model

In [ ]:
with torch.no_grad():
    # get some random training images
    dataiter = iter(test_loader)
    images, labels = dataiter.next()

    # show images
    imshow(torchvision.utils.make_grid(images))

    images = images.reshape(-1, 28*28)
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)




    # print labels
    print("ground truth: ", labels)
    print("predicted: ", predicted)

In [ ]:
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct.item() / total))